# RAG 파이프라인 구축하기

## 1. 사전작업(Pre-processing) - 1~4 단계

사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행합니다.

- 1단계 문서로드(Document Load): 문서 내용을 불러옵니다.
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할합니다.
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장합니다.
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장합니다.

## 2. RAG 수행(RunTime) - 5~8 단계

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다. Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
- 7단계 LLM: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다.

## * 실습에 활용할 문서

각자 1차 과제때 구상한 서비스를 구현하기 위해 필요한 문서를 구비하여 업로드 해주세요.

예시문서
- 문서명: <간이투자설명서> 삼성KODEX 코스피증권상장지수투자신탁[주식](펀드 코드: B6290)
- 링크: https://www.samsungfund.com/etf/product/library/pdf.do
- 파일명: `2ETF52-Aa`

_실습을 위해 다운로드 받은 파일을 구글 Colab의 `/content` 폴더로 복사해 주시기 바랍니다_

## * 뼈대 코드 활용

아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다. 각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [1]:
## 환경설정 : 필요한 라이브러리들을 설정합니다.

!pip install python-dotenv
!pip install faiss-cpu
!pip install pymupdf
!pip install langchain_community
!pip install langchain_openai


In [2]:
# 아래를 참고하여 생성한 API 키를 환경변수로 관리하기 위한 .txt 파일을 만들어 보세요
# txt 파일에 OPENAI_API_KEY=your-openai-api-key-here(발급받은 api key)를 붙혀넣기 하고.'env'라는 이름으로 저장한 뒤, /content에 파일을 업로드 해주세요

from dotenv import load_dotenv
import openai
import os

# API 키 정보 로드
load_dotenv()

# .env파일 생성 및 저장
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

In [3]:
## RAG 기본 파이프라인(1~8단계)

#langchain 라이브러리에서 필요한 기능들을 불러와 줍니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [5]:
# 단계 1: 문서 로드(Load Documents)
# 구현하고자 하는 서비스에 맞게, 직접 구비한 문서를 업로드 해주세요. .꼭 .pdf파일이 아니어도 좋습니다.
loader = PyMuPDFLoader("./content/011.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

"""페이지의 내용을 출력합니다."""

print(docs[1].page_content)

"""`metadata` 를 확인합니다."""

docs[1].__dict__

문서의 페이지수: 8
파이썬 기반의 AI 프로그래밍 01-1
- 2 -
다. 예측 분석(Predictive Analytics)
라. 통계적 머신러닝(Statistical Learning)
- 데이터에서 지식을 추출하는 작업
3) 머신러닝으로 풀 수 있는 문제
- 이미 알려진 사례를 바탕으로 일반화된 모델을 만들어 의사 결정 프로세스 자동
화 → 지도 학습(Supervised Learning)


{'id': None,
 'metadata': {'producer': 'Hancom PDF 1.3.0.443',
  'creator': 'Hancom PDF 1.3.0.443',
  'creationdate': '2020-11-05T23:33:03+09:00',
  'source': './content/011.pdf',
  'file_path': './content/011.pdf',
  'total_pages': 8,
  'format': 'PDF 1.4',
  'title': '제1항표준어는 교양 있는 사람들이 두루 쓰는 현대 서울말로 정함을 원칙으로 한다',
  'author': 'user',
  'subject': '',
  'keywords': '',
  'moddate': '2020-11-05T23:33:03+09:00',
  'trapped': '',
  'modDate': "D:20201105233303+09'00'",
  'creationDate': "D:20201105233303+09'00'",
  'page': 1},
 'page_content': '파이썬 기반의 AI 프로그래밍 01-1\n- 2 -\n다. 예측 분석(Predictive Analytics)\n라. 통계적 머신러닝(Statistical Learning)\n- 데이터에서 지식을 추출하는 작업\n3) 머신러닝으로 풀 수 있는 문제\n- 이미 알려진 사례를 바탕으로 일반화된 모델을 만들어 의사 결정 프로세스 자동\n화 → 지도 학습(Supervised Learning)',
 'type': 'Document'}

In [6]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 9


In [7]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [8]:
# 단계 4: DB 생성(Create DB) 및 저장
import faiss

# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

for doc in vectorstore.similarity_search("투자"):
    print(doc.page_content)

파이썬 기반의 AI 프로그래밍 01-1
- 4 -
파이썬 기반의 AI 프로그래밍 01-1
- 2 -
다. 예측 분석(Predictive Analytics)
라. 통계적 머신러닝(Statistical Learning)
- 데이터에서 지식을 추출하는 작업
3) 머신러닝으로 풀 수 있는 문제
- 이미 알려진 사례를 바탕으로 일반화된 모델을 만들어 의사 결정 프로세스 자동
화 → 지도 학습(Supervised Learning)
파이썬 기반의 AI 프로그래밍 01-1
- 3 -
- 입력은 주어지지만 출력은 제공되지 않음 → 비지도 학습(Unsupervised 
Learning)
파이썬 기반의 AI 프로그래밍 01-1
- 5 -
- 컴퓨터가 인식할 수 있는 형태의 입력 데이터 준비 필수
- sample, data point: 하나의 개체, 행
- feature: 샘플의 속성, 열
4) 문제와 데이터 이해하기


In [10]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

"""검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다."""

# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("핵심 내용은?")

[Document(id='deebeb70-d3af-4ad6-a0b3-a12c8191d75a', metadata={'producer': 'Hancom PDF 1.3.0.443', 'creator': 'Hancom PDF 1.3.0.443', 'creationdate': '2020-11-05T23:33:03+09:00', 'source': './content/011.pdf', 'file_path': './content/011.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': '제1항표준어는 교양 있는 사람들이 두루 쓰는 현대 서울말로 정함을 원칙으로 한다', 'author': 'user', 'subject': '', 'keywords': '', 'moddate': '2020-11-05T23:33:03+09:00', 'trapped': '', 'modDate': "D:20201105233303+09'00'", 'creationDate': "D:20201105233303+09'00'", 'page': 0}, page_content='파이썬 기반의 AI 프로그래밍 01-1\n- 1 -\n1. 머신러닝 개요\n1강. 머신러닝과 필수 라이브러리 \n  \n학습내용  \n - 인공지능과 머신러닝\n - 머신러닝 필수 라이브러리\n  \n학습목표  \n- 인공지능과 머신러닝의 개념을 설명할 수 있다.\n- 머신러닝의 필수 라이브러리를 열거할 수 있다.\n1. 인공지능과 머신러닝\n1) 인공지능\n가. 대량의 데이터에서 산출되는 어떤 특징적인 기능\n나. 데이터 바탕으로 특수한 패턴 발견 -> 취할 수 있는 최적의 행동 결정\n2) 머신러닝\n가. 데이터에서 지식을 추출하는 작업\n나. 통계학, 인공지능 및 컴퓨터 과학의 연구 분야'),
 Document(id='a5d4ef58-3722-4250-af68-93f21c99692d', metadata={'producer': 'Hancom PDF 1.3.0.443', 'creator': 

In [15]:
# 단계 6: 프롬프트 입력
# 2주차 과제에서 생성한 프롬프트를 여기에 입력해 주세요. 2주차 과제 피드백을 보고 zero-shot, few-shot, 후카츠식 프롬프트, 동적(dynamic) 프롬프팅 등 다양한 프롬프팅 기법을 적용해 보세요.
prompt = PromptTemplate.from_template(
    """You are a financial advisor chatbot.
Provide information about financial products, investment strategies,
and personal finance management.

answer korean

#Context:
{context}

#Question:
{question}

#Answer:"""
)

In [16]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0) #2주차 과제와 마찬가지로 여러가지 모델을 연동할 수 있습니다. OpenAI뿐 아니라 Gemini나 Claude 등 다양한 모델을 연동해 보셔도 좋습니다.

# 단계 8: 체인(Chain) 생성
# promt, llm, vectodDB를 이어붙인 간단한 chain입니다. 제공드린 핵심예제를 보고 기능에 맞게 추가로 다른 체인을 연동해 보세요.
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

"""생성된 체인에 쿼리(질문)을 입력하고 실행합니다."""

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "강의 자료의 핵심 내용은?"
response = chain.invoke(question)
print(response)

강의 자료의 핵심 내용은 인공지능과 머신러닝의 개념을 설명하고, 머신러닝의 필수 라이브러리를 소개하는 것입니다. 구체적으로는 인공지능과 머신러닝의 정의, 예측 분석과 통계적 머신러닝의 역할, 지도 학습과 비지도 학습의 차이점, 그리고 머신러닝을 위한 필수 라이브러리인 파이썬, 싸이킷런, 주피터 노트북, NumPy, SciPy, matplotlib, pandas, mglearn 등을 다루고 있습니다.
